### Zigbang 원룸 매물 데이터 수집

In [1]:
import requests
import pandas as pd

#### Process
    - 동이름으로 위도 경도 구하기
    - 위도 경도로 geohash 알아내기
    - geohash로 매물 아이디 가져오기
    - 매물 아이디로 매물 정보 가져오기

#### 1. 동이름으로 위도 경도 구하기

In [9]:
addr = '망원동'
url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'

response = requests.get(url)
response.text

'{"success":true,"code":"200","items":[{"id":4002,"type":"address","name":"망원동","hint":"","description":"서울특별시 마포구 망원동","lat":37.556785583496094,"lng":126.9013442993164,"zoom":5,"polygon":[],"_score":null,"_source":{"name_length":3,"local1":"서울시","local2":"마포구","local3":"망원동","web_level":15,"web_lat":37.556785583496094,"web_lng":126.9013442993164,"app_level":15,"app_lat":37.556785583496094,"app_lng":126.9013442993164,"법정동코드":"1144012300"},"zoom_level":{"google":15,"daum":4},"zoom_level_v2":{"app":5,"web":4}},{"id":156,"type":"school","name":"서울동교초등학교","hint":"","description":"서울특별시 마포구 망원동","lat":37.558210071,"lng":126.90235496,"zoom":3,"polygon":[],"_score":null,"_source":{"name_length":8,"school_code":"B100001977","gender":"both","stage":"elementary","sullib":"public","foundation_date":"1974-09-01","address":"서울시 마포구 망원동 426-5","local1":"서울특별시","local2":"마포구","local3":"망원동","suggestions_insensitive":["서울동교초등학교"],"address_road":"서울시 마포구 월드컵로25길 86","suggestions":["서울시마포구망원동426-5","서울시

In [12]:
data = response.json()['items'][0]
lat, lng = data['lat'],data['lng']
lat, lng

(37.556785583496094, 126.9013442993164)

#### 2. 위도 경도로 geohash 알아내기

In [14]:
!pip install geohash2

In [16]:
import geohash2

In [19]:
geohash =geohash2.encode(lat, lng, precision = 5)
geohash

'wydjx'

In [21]:
url = f'https://apis.zigbang.com/v2/items/oneroom?geohash={geohash}&depositMin=0&rentMin=0&salesTypes[0]=전세&salesTypes[1]=월세&domain=zigbang&checkAnyItemWithoutFilter=true'

response = requests.get(url)
response

<Response [200]>

380

#### 3. geohash로 매물 아이디 가져오기

In [ ]:
item_ids = []
for data in response.json()['items']:
    item_ids.append(data['itemId'])
len(item_ids)

In [ ]:
item_ids = [data['itemId'] for data in response.json()['items']]

#### 4. 매물 아이디로 매물 정보 가져오기

In [31]:
url = 'https://apis.zigbang.com/v2/items/list'

params = {
    'domain': "zigbang",
    'item_ids' : item_ids
}

reponse = requests.post(url, params)
reponse

<Response [200]>

In [38]:
data = reponse.json()['items']
df = pd.DataFrame(data)
df = df[['item_id', 'sales_title', 'deposit', 'rent', 'size_m2', 'floor', 'building_floor', 'title', 'manage_cost', 'address1']]
df.tail(2)
# df.columns

,item_id,sales_title,deposit,rent,size_m2,floor,building_floor,title,manage_cost,address1
378,42176516,월세,1000,75,37.00,반지하,2,반려동물O 깔끔한 투룸,3,서울시 마포구 성산동
379,42218289,월세,1000,49,15.84,2,5,가좌역2분 깔끔 엘베있는 필로티 원룸,10,서울시 마포구 중동


In [34]:
def oneroom(addr):
    url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
    response = requests.get(url)
    data = response.json()['items'][0]
    lat, lng = data['lat'],data['lng']
    lat, lng
    

Index(['item_id', 'sales_type'], dtype='object')